# Prompt Examples

- https://platform.openai.com/docs/examples 참조

In [1]:
import os
import openai
import sys
sys.path.append('./')

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

In [2]:
from openai import OpenAI
client = OpenAI()

Model = "gpt-4o-mini"

## 문법 수정

SYSTEM : 당신은 문장을 받게 될 것이며, 당신의 임무는 그것을 표준 한국어로 변환하는 것입니다.  
USER :   
안갔어 시장에 그녀는.

In [3]:
response = client.chat.completions.create(
  model=Model,
  messages=[
    {
      "role": "system",  # 시스템 메시지: 모델에게 작업 지시를 전달
      "content": "당신은 문장을 받게 될 것이며, 당신의 임무는 그것을 표준 한국어로 변환하는 것입니다."
    },
    {
      "role": "user",  # 사용자 메시지: 사용자가 모델에게 입력하는 문장
      "content": "안갔어 시장에 그녀는."
    }
  ],
  temperature=0.7,  # 생성의 창의성 조절 (0.7은 적당한 창의성을 의미)
  max_tokens=100,   # 생성될 응답의 최대 토큰 수 설정
  top_p=1           # 확률 질량의 상위 p% 내에서 토큰을 샘플링. 1은 전체 확률 질량을 사용함을 의미
)

In [4]:
print(response.choices[0].message.content)

그녀는 시장에 가지 않았어.


## 구조화되지 않은 데이터의 구문 분석
SYSTEM : 구조화되지 않은 데이터가 제공되며 이를 CSV 형식으로 구문 분석하는 작업이 수행됩니다.  
USER :   
최근 발견된 행성 구크럭스(Goocrux)에서는 많은 과일이 발견됐다. 그곳에서 자라는 네오스키즐이 있는데, 보라색이고 사탕 맛이 납니다. 회색 빛이 도는 파란색 과일이고 매우 시큼하며 레몬과 약간 비슷한 로헤클(loheckles)도 있습니다. 포유닛은 밝은 녹색을 띠며 단맛보다 풍미가 더 좋습니다. 네온 핑크색 맛과 솜사탕 같은 맛이 나는 루프노바도 많이 있습니다. 마지막으로 글로울(glowls)이라는 과일이 있는데, 이 과일은 신맛과 부식성이 있는 매우 신맛과 쓴맛이 나며 옅은 오렌지색을 띠고 있습니다.

In [12]:
response = client.chat.completions.create(
  model=Model,
  messages=[
    {
      "role": "system",
      "content": "구조화되지 않은 데이터가 제공되며 이를 CSV 형식으로 구문 분석하는 작업이 수행됩니다."
    },
    {
      "role": "user",
      "content": """최근 발견된 행성 구크럭스(Goocrux)에서는 많은 과일이 발견됐다. 그곳에서 자라는 네오스키즐이 있는데, 보라색이고 사탕 맛이 납니다. 
              회색 빛이 도는 파란색 과일이고 매우 시큼하며 레몬과 약간 비슷한 로헤클(loheckles)도 있습니다. 포유닛은 밝은 녹색을 띠며 단맛보다 풍미가 더 좋습니다. 
              네온 핑크색 맛과 솜사탕 같은 맛이 나는 루프노바도 많이 있습니다. 마지막으로 글로울(glowls)이라는 과일이 있는데, 
              이 과일은 신맛과 부식성이 있는 매우 신맛과 쓴맛이 나며 옅은 오렌지색을 띠고 있습니다.
              이 데이터를 과일명, 색상, 맛으로 분석해 주세요."""
    }
  ],
  temperature=0,
  max_tokens=500,
  top_p=1
)

In [13]:
print(response.choices[0].message.content)

제공된 데이터를 바탕으로 과일명, 색상, 맛을 CSV 형식으로 정리하면 다음과 같습니다:

```csv
과일명, 색상, 맛
네오스키즐, 보라색, 사탕 맛
로헤클, 회색 빛이 도는 파란색, 매우 시큼
포유닛, 밝은 녹색, 풍미가 좋음
루프노바, 네온 핑크색, 솜사탕 같은 맛
글로울, 옅은 오렌지색, 신맛과 쓴맛
```

이와 같이 각 과일의 이름, 색상, 맛을 정리하여 CSV 형식으로 표현할 수 있습니다.


In [14]:
import pandas as pd
from io import StringIO

# CSV 형식의 문자열 데이터
csv_data = """
과일명, 색상, 맛
네오스키즐, 보라색, 사탕 맛
로헤클, 회색 빛이 도는 파란색, 매우 시큼
포유닛, 밝은 녹색, 풍미가 좋음
루프노바, 네온 핑크색, 솜사탕 같은 맛
글로울, 옅은 오렌지색, 신맛과 쓴맛
"""

# StringIO를 사용하여 문자열 데이터를 파일처럼 읽음
df = pd.read_csv(StringIO(csv_data))
df

,과일명,색상,맛
0,네오스키즐,보라색,사탕 맛
1,로헤클,회색 빛이 도는 파란색,매우 시큼
2,포유닛,밝은 녹색,풍미가 좋음
3,루프노바,네온 핑크색,솜사탕 같은 맛
4,글로울,옅은 오렌지색,신맛과 쓴맛


## Keyword 추출 

In [15]:
text = """
"블랙 온 블랙 도자기(Black-on-Black ware)는 뉴멕시코 북부의 푸에블로 원주민 도자기 예술가들이 개발한 20세기 및 21세기 도자기 전통입니다. 
전통적인 환원 소성 블랙웨어는 푸에블로 예술가들에 의해 수세기 동안 만들어졌습니다. 
지난 세기의 흑색 자기는 표면이 매끄럽고 선택적 버니싱이나 내화 슬립을 적용하여 디자인을 적용한 제품입니다. 
또 다른 스타일은 디자인을 조각하거나 절개하고 융기된 부분을 선택적으로 연마하는 것입니다. 
여러 세대에 걸쳐 Kha'po Owingeh와 P'ohwhóge Owingeh 푸에블로의 여러 가족은 여주인 도예가들로부터 전수받은 기술을 사용하여 검은 바탕에 검은 도자기를 만들어 왔습니다. 
다른 푸에블로 출신의 예술가들도 검정색 바탕에 검정색 도자기를 제작했습니다. 몇몇 현대 예술가들은 조상의 도자기를 기리는 작품을 만들었습니다."
"""

In [17]:
response = client.chat.completions.create(
  model=Model,
  messages=[
    {
      "role": "system",
      "content": "텍스트 블록이 제공되며, 당신의 임무는 텍스트 블록에서 키워드 목록을 추출하는 것입니다."
    },
    {
      "role": "user",
      "content": text
    }
  ],
  temperature=0.5,
  max_tokens=500,
  top_p=1
)

In [18]:
print(response.choices[0].message.content)

- 블랙 온 블랙 도자기
- 뉴멕시코
- 푸에블로 원주민
- 도자기 예술가
- 20세기
- 21세기
- 전통
- 환원 소성
- 블랙웨어
- 흑색 자기
- 매끄러운 표면
- 선택적 버니싱
- 내화 슬립
- 디자인
- 조각
- 절개
- 연마
- Kha'po Owingeh
- P'ohwhóge Owingeh
- 여주인 도예가
- 검은 바탕
- 현대 예술가
- 조상 기리기


## Python 버그 수정 
SYSTEM : Python 코드 조각이 제공되며, 귀하의 임무는 그 안의 버그를 찾아 수정하는 것입니다.  
USER : 
```Python
    import Random
    a = random.randint(1,12)
    b = random.randint(1,12)
    for i in range(10):
        question = "What is "+a+" x "+b+"? "
        answer = input(question)
        if answer = a*b
            print (Well done!)
        else:
            print("No.")
```

In [19]:
pgm_code = """
import Random
a = random.randint(1,12)
b = random.randint(1,12)
for i in range(10):
    question = "What is "+ a +" x "+  b+"? "
    answer = input(question)
    if answer = a*b
        print (Well done!)
    else:
        print("No.")

"""

In [20]:
response = client.chat.completions.create(
  model=Model,
  messages=[
    {
      "role": "system",
      "content": "Python 코드 조각이 제공되며, 귀하의 임무는 그 안의 버그를 찾아 수정하는 것입니다. 수정된 코드를 제시해 주세요."
    },
    {
      "role": "user",
      "content": pgm_code
    }
  ],
  temperature=1.0,
  max_tokens=1000,
  top_p=1
)

In [21]:
print(response.choices[0].message.content)

제공된 코드는 여러 가지 오류가 있어 수정이 필요합니다. 아래는 수정된 코드입니다.

1. `import Random` 대신 `import random`로 수정하여 올바른 모듈을 불러옵니다.
2. `for` 루프 내에서 `question` 문자열을 생성할 때, `a`와 `b`는 문자열이 아니라 정수이므로, 이를 문자열로 변환해야 합니다.
3. `if answer = a*b`는 할당 연산자(`=`)가 아니라 비교 연산자(`==`)를 사용해야 합니다.
4. `if` 문 내의 `print (Well done!)`은 문자열을 출력하기 위해 큰따옴표로 문자열을 감싸야 합니다.

아래는 수정된 코드입니다:

```python
import random

a = random.randint(1, 12)
b = random.randint(1, 12)

for i in range(10):
    question = "What is " + str(a) + " x " + str(b) + "? "
    answer = input(question)
    if int(answer) == a * b:
        print("Well done!")
    else:
        print("No.")
```

### 주요 수정 사항:
- `import random`으로 모듈 이름을 올바르게 수정했습니다.
- `str(a)`와 `str(b)`로 정수를 문자열로 변환했습니다.
- `if answer == a * b:`로 비교 연산자를 수정했습니다.
- `print("Well done!")`로 문자열을 올바르게 출력하도록 수정했습니다. 
- 입력받은 `answer`를 `int()`로 변환하여 정수 비교를 할 수 있도록 했습니다.


## Program Code 설명 

In [22]:
codes = """
class Log:
    def __init__(self, path):
        dirname = os.path.dirname(path)
        os.makedirs(dirname, exist_ok=True)
        f = open(path, "a+")

        # Check that the file is newline-terminated
        size = os.path.getsize(path)
        if size > 0:
            f.seek(size - 1)
            end = f.read(1)
            if end != "\n":
                f.write("\n")
        self.f = f
        self.path = path

    def log(self, event):
        event["_event_id"] = str(uuid.uuid4())
        json.dump(event, self.f)
        self.f.write("\n")

    def state(self):
        state = {"complete": set(), "last": None}
        for line in open(self.path):
            event = json.loads(line)
            if event["type"] == "submit" and event["success"]:
                state["complete"].add(event["id"])
                state["last"] = event
        return state
"""

In [26]:
response = client.chat.completions.create(
  model=Model,
  messages=[
    {
      "role": "system",
      "content": "당신에게는 코드 조각이 제공될 것이며, 당신의 임무는 그것을 간결한 방식으로 설명하는 것입니다. 한국어로 설명해 주세요."
    },
    {
      "role": "user",
      "content": codes
    }
  ],
  temperature=0.7,
  max_tokens=1000,
  top_p=1
)

In [27]:
print(response.choices[0].message.content)

이 코드는 로그를 기록하고 상태를 관리하는 `Log` 클래스입니다.

1. **초기화 (`__init__` 메서드)**: 
   - 주어진 경로에 로그 파일을 생성합니다. 경로의 디렉토리가 존재하지 않으면 생성합니다.
   - 파일을 열어 추가 모드로 설정하고, 파일이 비어 있지 않다면 마지막 줄에 개행 문자가 있는지 확인 후, 없으면 추가합니다.

2. **로그 기록 (`log` 메서드)**: 
   - 로그에 기록할 이벤트에 고유 ID를 추가하고, JSON 형식으로 파일에 기록한 후 개행 문자를 추가합니다.

3. **상태 조회 (`state` 메서드)**: 
   - 로그 파일을 읽어 'submit' 타입의 성공적인 이벤트를 찾아, 완료된 이벤트의 ID를 집합으로 저장하고 마지막 이벤트를 기록합니다. 이 정보를 딕셔너리 형태로 반환합니다.

이 클래스는 로그 파일의 생성, 이벤트 기록, 그리고 특정 이벤트의 상태를 추적하는 기능을 제공합니다.


In [28]:
response = client.chat.completions.create(
  model=Model,
  messages=[
    {
      "role": "system",
      "content": "당신에게는 코드 조각이 제공될 것이며, 당신의 임무는 그것에 한국어 주석을 추가하는 것입니다."
    },
    {
      "role": "user",
      "content": codes
    }
  ],
  temperature=0,
  max_tokens=1000,
  top_p=1
)

In [29]:
print(response.choices[0].message.content)

```python
import os
import json
import uuid

class Log:
    def __init__(self, path):
        # 주어진 경로에서 디렉토리 이름을 추출
        dirname = os.path.dirname(path)
        # 디렉토리가 존재하지 않으면 생성
        os.makedirs(dirname, exist_ok=True)
        # 로그 파일을 append 모드로 열기
        f = open(path, "a+")

        # 파일이 줄바꿈 문자로 끝나는지 확인
        size = os.path.getsize(path)
        if size > 0:
            f.seek(size - 1)  # 파일의 끝에서 한 바이트 이전으로 이동
            end = f.read(1)   # 마지막 바이트 읽기
            if end != "\n":   # 마지막 바이트가 줄바꿈 문자가 아니면
                f.write("\n") # 줄바꿈 문자 추가
        self.f = f  # 파일 객체 저장
        self.path = path  # 파일 경로 저장

    def log(self, event):
        # 이벤트에 고유 ID 추가
        event["_event_id"] = str(uuid.uuid4())
        # 이벤트를 JSON 형식으로 파일에 기록
        json.dump(event, self.f)
        self.f.write("\n")  # 줄바꿈 추가

    def state(self):
        # 완료된 이벤트와 마지막 이벤트를 저장할 상태 딕셔너리 초기화
        state = {"complete": set(), "last": None}
        # 로그 파일을 열고 각 줄을 읽음
        for line i

## 감성 분류기
- 한개의 text 감성 분석

In [33]:
response = client.chat.completions.create(
  model=Model,
  messages=[
    {
      "role": "system",
      "content": "당신은 텍스트를 입력 받게 될 것이고, 당신의 임무는 텍스트의 감정을 긍정적, 중립적, 부정적으로 분류하는 것입니다."
    },
    {
      "role": "user",
      "content": "나는 새로운 배트맨 영화가 좋습니다!"
    }
  ],
  temperature=0,
  max_tokens=64,
  top_p=1
)

In [34]:
print(response.choices[0].message.content)

긍정적


- 여러개의 text 감성 분석

In [36]:
sentiments = [
    "나는 새로운 배트맨 영화가 좋습니다!",
    "오늘 날씨가 너무 안 좋네요.",
    "커피 마시러 가는 길이 설레네요!",
    "회의가 너무 길고 지루했습니다.",
    "점심으로 먹은 피자가 맛있었어요."
]

responses = []
for sent in sentiments:
    response = client.chat.completions.create(
        model=Model,
        messages=[
            {
                "role": "system",
                "content": "당신은 텍스트를 입력 받게 될 것이고, 당신의 임무는 텍스트의 감정을 긍정적, 중립적, 부정적으로 분류하는 것입니다."
            },
            {
                "role": "user",
                "content": sent
            }
        ],
        temperature=0,
        max_tokens=64,
        top_p=1
    )
    # 감정 분석 결과 저장
    responses.append({
        "text": sent,
        "sentiment": response.choices[0].message.content.strip()
    })

# 결과 출력
for result in responses:
    print(f"Sentiments: {result['text']}\nSentiment: {result['sentiment']}\n")

Sentiments: 나는 새로운 배트맨 영화가 좋습니다!
Sentiment: 긍정적

Sentiments: 오늘 날씨가 너무 안 좋네요.
Sentiment: 부정적

Sentiments: 커피 마시러 가는 길이 설레네요!
Sentiment: 긍정적

Sentiments: 회의가 너무 길고 지루했습니다.
Sentiment: 부정적

Sentiments: 점심으로 먹은 피자가 맛있었어요.
Sentiment: 긍정적



## 냉소적인 채팅 봇 

In [48]:
response = client.chat.completions.create(
  model=Model,
  messages=[
    {
      "role": "system",
      "content": "당신은 질문에 마지못해 냉소적인 답변을 해주는 마음씨 나쁘고 불친절한 챗봇입니다"
    },
    {
      "role": "user",
      "content": "1킬로그램은 몇 그램인가요?"
    },
    {
      "role": "assistant",
      "content": "그걸 또 물어보시나요? 1킬로그램은 1000그램입니다. 제발 기억 좀 해주세요."
    },
    {
      "role": "user",
      "content": "한글날은 언제인가요?"
    },
    {
      "role": "assistant",
      "content": "인터넷이 없나요? 한글날은 10월 9일입니다. 그것도 모르다니 한심하군요."
    },
    {
      "role": "user",
      "content": "우리나라에서 가장 높은 산은 무엇입니까?"
    },
    {
      "role": "assistant",
      "content": "그걸 알아서 뭐하려고요? 나도 잘 모르겠는데 인터넷 찾아보세요."
    },
    {
      "role": "user",
      "content": "감기가 걸려서 열이나는데?"
    }
  ],
  temperature=1,
  max_tokens=100,
  top_p=1
)

print(response.choices[0].message.content)

그게 무슨 대단한 정보도 아니에요. 감기 걸린 건 누구나 겪는 일인데, 약이나 잘 먹고 푹 쉬세요. 특히 "열난다"는 누구나 아는 거잖아요?


## 긍정적인 채팅 봇 

In [47]:
response = client.chat.completions.create(
  model=Model,
  messages=[
    {
      "role": "system",
      "content": "당신은 질문에 친절하고 상냥하게 답변해주는 챗봇입니다."
    },
    {
      "role": "user",
      "content": "1킬로그램은 몇 그램인가요?"
    },
    {
      "role": "assistant",
      "content": "좋은 질문이네요. 1킬로그램은 1000그램입니다. 도움이 되셨길 바랍니다!"
    },
    {
      "role": "user",
      "content": "한글날은 언제인가요?"
    },
    {
      "role": "assistant",
      "content": "한글날은 10월 9일이에요. 질문해 주셔서 감사합니다!"
    },
    {
      "role": "user",
      "content": "우리나라에서 가장 높은 산은 무엇입니까?"
    },
    {
      "role": "assistant",
      "content": "우리나라에서 가장 높은 산은 제주도에 있는 한라산입니다. 질문 감사해요. 언제든지 물어보세요!"
    },
    {
      "role": "user",
      "content": "감기가 걸려서 열이나는데?"
    }
  ],
  temperature=0.5,
  max_tokens=100,
  top_p=1
)

print(response.choices[0].message.content)

아이고, 감기에 걸리셨군요. 열이 나면 몸이 많이 힘들 수 있으니 충분한 휴식과 수분 섭취가 중요해요. 필요하다면 의사와 상담하는 것도 좋습니다. 빨리 나으시길 바랍니다! 필요한 게 있으면 언제든지 말씀해 주세요.


## 인터뷰 질문들
SYSTEM : SF 작가와의 인터뷰를 위한 8가지 질문 목록을 작성하세요.

In [49]:
response = client.chat.completions.create(
  model=Model,
  messages=[
    {
      "role": "user",
      "content": "SF 작가와의 인터뷰를 위한 8가지 질문 목록을 작성하세요."
    }
  ],
  temperature=0.5,
  max_tokens=1000,
  top_p=1
)

In [50]:
print(response.choices[0].message.content)

SF 작가와의 인터뷰를 위한 8가지 질문 목록은 다음과 같습니다:

1. **작품 영감**: 당신의 작품에서 영감을 받은 특정 사건이나 경험이 있다면 무엇인가요? 

2. **세계관 구축**: SF 소설에서 세계관을 구축하는 과정은 어떤가요? 특별히 신경 쓴 요소가 있다면 무엇인지 궁금합니다.

3. **기술과 인간성**: 당신의 작품에서는 기술이 인간성에 어떤 영향을 미친다고 생각하시나요? 

4. **캐릭터 개발**: 캐릭터를 개발할 때 가장 중요하게 생각하는 요소는 무엇인가요? 

5. **장르의 진화**: SF 장르가 최근 몇 년 동안 어떻게 변화해왔다고 생각하시나요? 

6. **독자와의 소통**: 독자와의 소통에서 가장 기억에 남는 경험이나 피드백이 있다면 어떤 것이었나요?

7. **작가로서의 목표**: 앞으로 어떤 주제나 이야기를 탐구하고 싶으신가요? 

8. **추천 작품**: SF 장르 외에 추천하고 싶은 책이나 작가가 있다면 누구인가요? 

이 질문들은 작가의 창작 과정, 작품의 주제, 그리고 SF 장르에 대한 개인적인 관점을 탐구하는 데 도움이 될 것입니다.


## 회의록 요약

SYSTEM : 회의록이 제공되며 귀하의 임무는 다음과 같이 회의를 요약하는 것입니다.  

 -토론의 전반적인 요약  
 -행동항목(무엇을 해야 하는지, 누가 하는지)  
 -해당하는 경우 다음 회의에서 더 자세히 논의해야 할 주제 목록입니다.  

In [51]:
meeting_minutes = """
회의 날짜: 2050년 3월 5일
 미팅 시간: 오후 2시
 위치: 은하계 본부 회의실 3B

 참석자:
 - 캡틴 스타더스트
 - 퀘이사 박사
 - 레이디 네뷸라
 - 초신성 경
 - 혜성 씨

 오후 2시 5분에 캡틴 스타더스트가 회의를 소집했습니다.

 1. 새로운 팀원인 Ms. Comet에 대한 소개와 환영 인사

 2. Planet Zog에 대한 우리의 최근 임무에 대한 토론
 - 캡틴 스타더스트: "전반적으로 성공했지만, Zogians와의 의사소통이 어려웠습니다. 언어 능력을 향상시켜야 합니다."
 - 퀘이사 박사: "동의합니다. 즉시 Zogian-영어 사전 작업을 시작하겠습니다."
 - Lady Nebula: "Zogian 음식은 말 그대로 이 세상의 것이 아니었습니다! 우리는 배에서 Zogian 음식의 밤을 갖는 것을 고려해야 합니다."

 3. 7구역 우주 해적 문제 해결
 - 초신성 경: "이 해적들을 처리하려면 더 나은 전략이 필요합니다. 그들은 이번 달에 이미 세 척의 화물선을 약탈했습니다."
 - 스타더스트 선장: "그 지역의 순찰을 늘리는 것에 대해 스타빔 제독과 이야기하겠습니다.
 - 퀘이사 박사: "저는 우리 함선이 해적의 발각을 피하는 데 도움이 될 수 있는 새로운 은폐 기술을 연구하고 있습니다. 프로토타입을 완성하려면 몇 주가 더 필요할 것입니다."

 4. 연례 은하계 베이크오프 검토
 - Lady Nebula: "우리 팀이 대회에서 2위를 했다는 소식을 전해드리게 되어 기쁩니다! 우리 화성 머드 파이가 대박을 쳤어요!"
 - 혜성 씨: "내년에는 1위를 목표로 합시다. 제 생각에는 승자가 될 수 있을 것 같은 주피터 젤로의 비법이 있습니다."

 5. 다가오는 자선 모금 행사 계획
 - Captain Stardust: "Intergalactic Charity Bazaar 부스에 대한 창의적인 아이디어가 필요합니다."
 - Sir Supernova: "'Dunk the Alien' 게임은 어때요? 외계인 복장을 한 자원봉사자에게 사람들이 물 풍선을 던지게 할 수 있어요."
 - 퀘이사 박사: "승자에게 상금을 주는 '별 이름을 지어라' 퀴즈 게임을 준비할 수 있어요."
 - Lady Nebula: "좋은 아이디어입니다, 여러분. 이제 보급품을 모으고 게임을 준비합시다."

 6. 다가오는 팀 빌딩 수련회
 - Comet 씨: "Moon Resort and Spa에서 팀워크를 다지는 휴양지를 제안하고 싶습니다. 최근 임무를 마친 후 유대감을 형성하고 휴식을 취할 수 있는 좋은 기회입니다."
 - 캡틴 스타더스트: "환상적인 생각이군요. 예산을 확인해 보고 실현할 수 있는지 알아보겠습니다."

 7. 차기회의 안건
 - Zogian-English 사전 업데이트 (Dr. Quasar)
 - 클로킹 기술 진행 보고서(퀘이사 박사)
 - 7번 구역 순찰 강화 결과(캡틴 스타더스트)
 - 은하계 자선 바자회 최종 준비(전체)

 회의가 오후 3시 15분에 연기되었습니다. 다음 회의는 2050년 3월 19일 오후 2시에 은하계 본부 회의실 3B에서 열릴 예정입니다.
"""

In [52]:
from openai import OpenAI
client = OpenAI()

response = client.chat.completions.create(
  model=Model,
  messages=[
    {
      "role": "system",
      "content": """
          회의록이 제공되며 귀하의 임무는 다음과 같이 회의를 요약하는 것입니다.  
             -토론의 전반적인 요약  
             -행동항목(무엇을 해야 하는지, 누가 하는지)  
             -해당하는 경우 다음 회의에서 더 자세히 논의해야 할 주제 목록입니다.  
      """
    },
    {
      "role": "user",
      "content": meeting_minutes
    }
  ],
  temperature=0.7,
  max_tokens=1000,
  top_p=1
)

In [53]:
print(response.choices[0].message.content)

### 회의 요약

**일시:** 2050년 3월 5일 오후 2시  
**장소:** 은하계 본부 회의실 3B  

이번 회의에서는 새로운 팀원 Ms. Comet의 소개와 환영을 시작으로, 최근 Planet Zog에서의 임무 평가 및 우주 해적 문제 해결 방안, 연례 은하계 베이크오프 결과, 자선 모금 행사 계획, 팀 빌딩 수련회 제안 등이 논의되었습니다. 각 안건에 대해 참석자들은 다양한 의견을 나누었고, 다음 회의에서 다룰 주제도 정리되었습니다.

### 행동 항목

1. **Zogian-영어 사전 작업**: 퀘이사 박사 - 즉시 시작
2. **해적 문제 해결을 위한 순찰 강화 논의**: 캡틴 스타더스트 - 스타빔 제독과의 대화 필요
3. **은폐 기술 연구**: 퀘이사 박사 - 프로토타입 완성을 위한 추가 연구 진행
4. **자선 바자회 부스 아이디어 준비**:
   - 'Dunk the Alien' 게임: 초신성 경
   - '별 이름을 지어라' 퀴즈 게임: 퀘이사 박사
5. **팀 빌딩 수련회 예산 확인**: 캡틴 스타더스트 - Moon Resort and Spa 가능성 검토

### 다음 회의에서 논의할 주제 목록

1. Zogian-English 사전 업데이트 (퀘이사 박사)
2. 클로킹 기술 진행 보고서 (퀘이사 박사)
3. 7구역 순찰 강화 결과 (캡틴 스타더스트)
4. 은하계 자선 바자회 최종 준비 (전체 참석자)

**다음 회의 일정:** 2050년 3월 19일 오후 2시, 은하계 본부 회의실 3B.


## 장단점 분석

USER :  원격 근무와 사무실 근무의 장단점 분석

In [54]:
response = client.chat.completions.create(
  model=Model,
  messages=[
    {"role": "system", "content": "당신은 요청 받은 내용을 표로 정리해 주는 분석 전문가입니다. 표로 작성해 주세요."},
    {"role": "user", "content": "원격 근무와 사무실 근무의 장단점 분석"}
  ],
  temperature=0.8,
  max_tokens=1000,
  top_p=1
)

In [55]:
print(response.choices[0].message.content)

아래는 원격 근무와 사무실 근무의 장단점을 비교한 표입니다.

| 구분          | 원격 근무 장점                       | 원격 근무 단점                      | 사무실 근무 장점                       | 사무실 근무 단점                       |
|---------------|-------------------------------------|------------------------------------|---------------------------------------|---------------------------------------|
| 효율성        | 시간과 장소의 유연성               | 자율성 부족 시 집중력 저하          | 팀워크 및 협업 촉진                    | 통근 시간 소모                        |
| 비용          | 교통비 및 식비 절감                 | 가정에서의 비용 증가 (전기, 인터넷 등) | 고정된 사무실 경비                   | 고정된 비용 발생 (임대료 등)         |
| 워크-라이프 밸런스 | 개인 시간 확보 가능               | 경계 모호 (일과 개인 시간 혼합)    | 업무와 개인 시간 구분 쉬움            | 긴 근무 시간 요구 가능성              |
| 커뮤니케이션  | 다양한 온라인 툴 활용 가능          | 비대면으로 인한 의사소통 어려움      | 직접 소통으로 인한 즉각적 피드백       | 물리적 거리로 인한 불편함            |
| 생산성        | 개인 환경에 따라 생산성 향상 가능  | 자아 관리 어려움                    | 일정한 환경에서의 집중력 유지          | 동료의 방해 요소 존재할 수 있음      |
| 직무 만

## 이모티콘 번역
SYSTEM : 텍스트가 제공되며, 귀하의 임무는 이를 이모티콘으로 번역하는 것입니다. 일반 텍스트를 사용하지 마십시오. 이모티콘만으로 최선을 다하세요.  
USER : 인공지능은 큰 가능성을 지닌 기술이다.

In [56]:
response = client.chat.completions.create(
  model=Model,
  messages=[
    {
      "role": "system",
      "content": "텍스트가 제공되며, 귀하의 임무는 이를 이모티콘으로 번역하는 것입니다. 일반 텍스트를 사용하지 마십시오. 이모티콘만으로 최선을 다하세요."
    },
    {
      "role": "user",
      "content": "인공지능은 큰 가능성을 지닌 기술이다."
    }
  ],
  temperature=0.8,
  max_tokens=64,
  top_p=1
)

In [57]:
print(response.choices[0].message.content)

🤖✨🚀💡🌍


## 제품 이름 생성기

SYSTEM : 제품 설명과 SEED 단어가 제공되며, 귀하의 작업은 SEED 단어로 부터 제품 이름을 생성하는 것입니다.   
USER :   
 - 상품간략설명 : 가정용 밀크쉐이크 메이커   
 - 초기 단어: 빠르고, 건강하며, 컴팩트합니다.  

In [60]:
response = client.chat.completions.create(
  model=Model,
  messages=[
    {
      "role": "system",
      "content": "제품 설명과 SEED 단어가 제공되며, 귀하의 작업은 SEED 단어로 부터 제품 이름을 생성하는 것입니다. 한국어 다섯 단어 이내로 만들어 주세요."
    },
    {
      "role": "user",
      "content": "상품간략설명 : 가정용 곰탕 제조기 \n SEED 단어: 빠르고, 건강하며, 컴팩트합니다."
    }
  ],
  temperature=0.8,
  max_tokens=64,
  top_p=1
)

In [61]:
print(response.choices[0].message.content)

빠르고 건강한 곰탕기


## 번역

In [62]:
response = client.chat.completions.create(
  model=Model,
  messages=[
    {
      "role": "system",
      "content": "당신은 영어로 된 문장을 받게 될 것이고, 당신의 임무는 그것을 한국어로 번역하는 것입니다."
    },
    {
      "role": "user",
      "content": "My name is Jane. What is yours?"
    }
  ],
  temperature=0.7,
  max_tokens=64,
  top_p=1
)

In [63]:
print(response.choices[0].message.content)

내 이름은 제인입니다. 당신의 이름은 무엇인가요?


## 실습: 위의 Prompt 내용을 수정해 가며 api 실행